# Wrangling PANSTEATITIS study ST001052
---

This notebook carries out the wrangling process for the [LIVER study ST001052 lipidomics data](https://www.metabolomicsworkbench.org/data/DRCCMetadata.php?Mode=Study&StudyID=ST001052&StudyType=MS&ResultType=1) [1].

## Libraries

In [1]:
using Pkg 

In [2]:
Pkg.activate(joinpath(@__DIR__, "..", ".."))

  Activating project at `~/git/gregfa/metabolomics/mlm-metabolomics-supplement/PANSTEATITISstudy`


In [60]:
Pkg.instantiate()

In [4]:
# To use RCall for the first time, one needs to 
# the location of the R home directory.
firstTimeRCall = false
if firstTimeRCall
    using Pkg
    io = IOBuffer()
    versioninfo(io)
    if occursin("Windows", String(take!(io)))
        ENV["R_HOME"] = "C:/PROGRA~1/R/R-43~1.1" # from R.home() in R
    else 
        ENV["R_HOME"] = "/usr/lib/R"

    end
    Pkg.build("RCall")
end      

In [5]:
using DataFrames, CSV
using FreqTables #, CategoricalArrays
using StatsBase
using RCall# Conda, PyCall
using MetabolomicsWorkbenchAPI

[ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414] (cache misses: incompatible header (6))


## Ext. Functions

In [6]:
include(joinpath(@__DIR__,"..","..","src","wrangling_utils.jl" ));
include(joinpath(@__DIR__,"..","..","src","demog.jl" ));

## Load data ST001052

In [7]:
ST  = "ST001052";

## Extract clinical covariates

Use the Julia's API to get the samples data from the [metabolomics workbench](https://www.metabolomicsworkbench.org/data/DRCCMetadata.php?Mode=Study&StudyID=ST001052).

In [8]:
# get clinical covariates
dfIndividuals =  fetch_samples(ST);
print_df_size(dfIndividuals)

The dataframe contains 53 rows and 17 columns


List of the covariate names: 

In [9]:
names(dfIndividuals)

17-element Vector{String}:
 "Sample ID"
 "Group"
 "Gender"
 "Date Captured"
 "Annuli"
 "Age"
 "WEIGHT (KG)"
 "LENGTH (CM)"
 "TG (CM)"
 "VET SCORE (Adipose)"
 "TOTAL PROTEIN (g/100mL)"
 "PCV Color"
 "PCV"
 "Histology Adipose"
 "Histology Liver"
 "Histology Swim Bladder"
 "NECROPSY NOTES:"

In [10]:
println("From the study description, $(ST) has $(fetch_total_subjects(ST)) subjects.")

From the study description, ST001052 has 51 subjects.


The clinical covariates dataframe contains 2 extra rows. We need to indicate what values corresponds to the `missing` data. In our case, all "-" will be replaced by `missing`.    

In [11]:
# assign missing value to "-"
dfIndividuals = ifelse.(dfIndividuals .== "-", missing, dfIndividuals);

Check number of missing per columns.

In [12]:
print_variables_missing(dfIndividuals)

Group contains 2 missing values.
Gender contains 2 missing values.
Date Captured contains 1 missing values.
Annuli contains 2 missing values.
Age contains 2 missing values.
WEIGHT (KG) contains 2 missing values.
LENGTH (CM) contains 2 missing values.
TG (CM) contains 2 missing values.
VET SCORE (Adipose) contains 16 missing values.
TOTAL PROTEIN (g/100mL) contains 3 missing values.
PCV Color contains 3 missing values.
PCV contains 3 missing values.
Histology Adipose contains 9 missing values.
Histology Liver contains 2 missing values.
Histology Swim Bladder contains 7 missing values.
NECROPSY NOTES: contains 14 missing values.


### Clinical dictionary

In [13]:
fileClinicalDict = joinpath(@__DIR__,"..","..","data","processed", "ClinicalDataDictionary.csv");
open(fileClinicalDict,"w") do io
   println(io,
        "Variable name, Variable description\n",
        "Gender,Sex\n",
        "Age, Years\n",
        "Weight (KG), Kilogram\n",
        "Length (CM), Centimeter\n",
        "Annuli,Number of opaque zones on fish scales\n",
        # "TG (CM),???\n",
        "VET SCORE,Veterinarian score where vet score < 1 indicates healthy tilapia and  vet score ≥ 1 indicates pansteatitis-affected tilapia.\n" ,
        "PCV Color,Pigmentation visually observed\n",
        "PCV,Pigmentation concentration volume\n",
        "Histology Adipose,Histological examination score of the adipose tissue\n",
        "Histology Liver,Histological examination score of the liver tissue\n",
        "Histology Swim Bladder,Histological examination score of the swim bladder tissue"
    )
end

### Independent variables

Select variables of interest:

In [14]:
select!(dfIndividuals, Symbol.(["Sample ID",
                                "Group",
                                "Gender",
                                "Annuli",
                                "Age",
                                "WEIGHT (KG)",
                                "LENGTH (CM)",
                                "Histology Adipose",
                                # "Histology Liver",
                                # "Histology Swim Bladder",
]));

Rename variables if needed:

In [15]:
rename!(dfIndividuals, Dict(
        :Group => "Status",
        :Gender => "Sex",
        Symbol("Sample ID") => "SampleID",
        Symbol("WEIGHT (KG)") => "Weight",
        Symbol("LENGTH (CM)") => "Length",
        Symbol("Histology Adipose") => "Histological_Score",
        # Symbol("Histology Liver") => "Histology_Liver",
        # Symbol("Histology Swim Bladder") => "Histology_Swim_Bladder",
));

#### Extract histology score   

The histology score spans between 0 and 5. Let write the histology dictionnary:   

In [16]:
fileHystologyDict = joinpath(@__DIR__,"..","..","data","processed", "HistologyDictionary.csv");
open(fileHystologyDict,"w") do io
   println(io,
        "Score, Histological Score Progression\n",
        "0, no signs\n",
        "1, minimal\n",
        "2, mild/few\n",
        "3, moderate",
        "4, moderate/severe\n",
        "5, severe\n",
    )
end

Create a function to extract each score:

In [17]:
function xtrcHistScore(vecHist) 
    idxnotmissing = findall(.!(ismissing.(vecHist)));
    vScore = Vector{Union{Missing, Int}}(undef, length(vecHist));
    for i in idxnotmissing
        if isdigit(vecHist[i][1])
           vScore[i] = parse(Int, vecHist[i][1]) 
        end
    end
    
    return vScore
end;

Replace original histology text by extracted score:

In [18]:
dfIndividuals.Histological_Score .= xtrcHistScore(dfIndividuals.Histological_Score);
# dfIndividuals.Histology_Liver .= xtrcHistScore(dfIndividuals.Histology_Liver);
# dfIndividuals.Histology_Swim_Bladder .= xtrcHistScore(dfIndividuals.Histology_Swim_Bladder);

Filter incomplete cases:

In [19]:
# filter complete cases
idxComplete = findall(completecases(dfIndividuals))
dfIndividuals = dfIndividuals[idxComplete, :]

# add a prefix to the ID samples
dfIndividuals.SampleID = "ID_".*string.(dfIndividuals.SampleID);

first(dfIndividuals, 5)

Row,SampleID,Status,Sex,Annuli,Age,Weight,Length,Histological_Score
,String,String?,String?,String?,String?,String?,String?,Int64?
1,ID_8358,FD,F,7,8,1.3,40,3
2,ID_8363,FD,F,12,13,1.5,40,5
3,ID_8370,FD,F,10,11,1.4,40.5,2
4,ID_8371,FD,F,11,12,1.4,41,2
5,ID_8373,FD,F,12,13,1.9,43.5,4


Insert a `GroupStatus` variable. The `Group` variable includes the diseases status and gender:

In [20]:
unique(dfIndividuals.Status)

4-element Vector{Union{Missing, String}}:
 "FD"
 "FH"
 "MD"
 "MH"

Let redefine the `Status` variable:

In [21]:
# insertcols!(dfIndividuals, 3, :GroupStatus => occursin.("D", dfIndividuals.Group));
idxDiseased = findall(occursin.("D", dfIndividuals.Status)) ;
idxHealthy = findall(occursin.("H", dfIndividuals.Status));

dfIndividuals.Status[idxDiseased] .= "Diseased";
dfIndividuals.Status[idxHealthy] .= "Healthy";

idxMale = findall(occursin.("M", dfIndividuals.Sex)) ;
idxFemale = findall(occursin.("F", dfIndividuals.Sex));

dfIndividuals.Sex[idxMale] .= "Male";
dfIndividuals.Sex[idxFemale] .= "Female";

In [22]:
first(dfIndividuals, 5)

Row,SampleID,Status,Sex,Annuli,Age,Weight,Length,Histological_Score
,String,String?,String?,String?,String?,String?,String?,Int64?
1,ID_8358,Diseased,Female,7,8,1.3,40,3
2,ID_8363,Diseased,Female,12,13,1.5,40,5
3,ID_8370,Diseased,Female,10,11,1.4,40.5,2
4,ID_8371,Diseased,Female,11,12,1.4,41,2
5,ID_8373,Diseased,Female,12,13,1.9,43.5,4


#### Save processed individuals dataset:

In [23]:
fileIndividuals = joinpath(@__DIR__,"..","..","data","processed","ST001052_ClinicalCovariates.csv");
dfIndividuals |> CSV.write(fileIndividuals);

### Demography

In [24]:
dfDemog = getDemographicST001052()

Row,Clinical Features,Count/ mean(SD)
,Any,Any
1,Status,
2,Diseased,30
3,Healthy,14
4,Sex,
5,Female,20
6,Male,24
7,Annuli,8.3(2.66)
8,Age,9.3(2.66)
9,WEIGHT (KG),1.67(0.31)


In [25]:
fileDemog = joinpath(@__DIR__,"..","..","data","processed","Demog.csv");
dfDemog |> CSV.write(fileDemog);

## Extract Metabolite references

In [26]:
# get clinical covariates
dfRef =  fetch_metabolites(ST);
print_df_size(dfRef)

The dataframe contains 590 rows and 10 columns


List the name of available properties:

In [27]:
names(dfRef)

10-element Vector{String}:
 "Metabolite"
 "quantified m/z"
 "rtimes"
 "ID_Ranked (LipidMatch OR LipidSearch annotation/rank)"
 "Class_At_Max_Intensity"
 "Adduct_At_Max_Intensity"
 "(LipidMatch Normalizer output)"
 "IS_Species"
 "IS_Adduct"
 "Neg & Pos Confirmed"

In [28]:
first(dfRef, 5)

Row,Metabolite,quantified m/z,rtimes,ID_Ranked (LipidMatch OR LipidSearch annotation/rank),Class_At_Max_Intensity,Adduct_At_Max_Intensity,(LipidMatch Normalizer output),IS_Species,IS_Adduct,Neg & Pos Confirmed
,String,String,String,String,String,String,String,String,String,String
1,CE(18:1),668.634144,15.04984231,1_CE(18:1)+NH4,CE,[M+NH4]+,1,CE(19:0),[M+NH4]+,No
2,CE(18:2),666.6186284,14.54040077,1_CE(18:2)+NH4,CE,[M+NH4]+,1,CE(19:0),[M+NH4]+,No
3,CE(18:3),664.6024729,14.00371744,1_CE(18:3)+NH4,CE,[M+NH4]+,1,CE(19:0),[M+NH4]+,No
4,CE(18:4),662.5874291,13.40464795,1_CE(18:4)+NH4,CE,[M+NH4]+,1,CE(19:0),[M+NH4]+,No
5,CE(20:1),696.6651802,15.52861207,1_CE(20:1)+NH4,CE,[M+NH4]+,1,CE(19:0),[M+NH4]+,No


Create a metabolite ID and keep only name and ID:

In [29]:
dfRef.MetaboliteID = "MT" .* string.(10000 .+ collect(1:size(dfRef, 1)));
select!(dfRef, [:Metabolite, :MetaboliteID]);

### Get Classification information

To get the classifciation information, we use the package `MetabolomicsWorkbenchAPI.jl`.

In [30]:
dfClassification = fetch_properties(dfRef.Metabolite);
# insertcols!(dfClassification, 1, :metabolite_name => dfRef.metabolite_name);
first(dfClassification, 3)

Row,exactmass,formula,main_class,refmet_name,sub_class,super_class
,String?,String?,String?,String?,String?,String?
1,650.6002,C45H78O2,Sterol esters,CE 18:1,Chol. esters,Sterol Lipids
2,648.5845,C45H76O2,Sterol esters,CE 18:2,Chol. esters,Sterol Lipids
3,646.5689,C45H74O2,Sterol esters,CE 18:3,Chol. esters,Sterol Lipids


In [31]:
dfRef |> names

2-element Vector{String}:
 "Metabolite"
 "MetaboliteID"

In [32]:
dfClassification |> names

6-element Vector{String}:
 "exactmass"
 "formula"
 "main_class"
 "refmet_name"
 "sub_class"
 "super_class"

In [33]:
# replace "-" by missing
for c ∈ eachcol(dfClassification)
           replace!(c, "-" => missing)
end

In [34]:
idxmissing = findall(ismissing.(dfClassification.main_class))
dfRef.Metabolite[idxmissing]

99-element Vector{String}:
 "OxLPC(20:2(OO))"
 "OxLPC(20:2(OOO))"
 "OxLPC(20:3(OH))"
 "OxLPC(20:4(OH))"
 "OxLPC(22:2(OO))"
 "OxLPC(22:2(OOO))"
 "OxLPC(22:3(OH))"
 "OxLPC(22:6(OOOO))"
 "OxPC(16:0_20:3(OH))"
 "OxPC(16:0_22:3(OH))"
 "OxPC(16:0_22:6(OO))"
 "OxPC(16:0_22:6(OOOO))"
 "OxPC(18:0_22:3(OH))"
 ⋮
 "plasmanyl-TG(O-20:1_16:0_18:1)"
 "plasmenyl-TG(P-20:1_15:0_16:0)"
 "plasmenyl-TG(P-20:1_16:0_18:0)"
 "plasmenyl-TG(P-20:1_16:1_18:1)"
 "PMe(16:0/18:1)"
 "TG(18:2+3O/16:0/17:0)"
 "TG(18:2+3O/17:0/22:5)"
 "TG(18:2+O/20:4/22:6)"
 "TG(18:3+3O/20:0/20:0)"
 "TG(22:5+O/22:4/22:6)"
 "TG(22:6+3O/18:1/19:0)"
 "ZyE(22:5)"

In [35]:
size(dfRef)

(590, 2)

To be able to use all the lipid, especially the Triglycerides, we need to adjust the name in a more standardized way to be able to extract their properties information.  
- The *Ox-* prefix mean oxidized, such as in *OxTG(16:0_20:5_20:3(OH))*. The *OH* indicates that it is a [TG hydroperoxide](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6550225/) [2].   
- The *P-* and the *O-* indicate respectively if the lipid is a plasmalogen or a plasmanyl, such as *plasmanyl-TG(O-20:0_18:0_18:4)* and *plasmenyl-TG(P-20:1_15:0_16:0)*[3].
- The non-oxidized lipids that contains Oxygen are described in [4].


In [36]:
dfRefOriginal = copy(dfRef);

In [37]:
dfRef = copy(dfRefOriginal);

In [38]:
dfRef.StandardizedName = copy(dfRef.Metabolite);

In [39]:
function standardizename(df::DataFrame, colname::String, matchstring)
    if !([colname] ⊆ names(df))
        df[:, colname] = repeat([false], size(dfRef, 1));
    end
    idx = findall(occursin.(matchstring, dfRef.Metabolite));
    df[idx, colname] .= true;
    # standardize name 
    df.StandardizedName[idx] .= replace.(dfRef.StandardizedName[idx], matchstring=>""); 
    return df
end

standardizename (generic function with 1 method)

In [40]:
newcolname = ["Oxidized", "OH", "O", "O₂", "O₃", "O₄", "Plasmanyl", "Plasmalogen",
              "O₂", "O₃", "Plasmanyl", "Plasmalogen", "CHO", "Ke", "O" ] 
rmvstring = ["Ox", "(OH)", "+O", "(OO)", "(OOO)", "(OOOO)", "O-", "P-",
             "+OO", "+3O", r"(?i)plasmanyl-", r"(?i)plasmenyl-", "(CHO)", "(Ke)", "O" ]
for i in 1:length(newcolname)
    dfRef = standardizename(dfRef, newcolname[i], rmvstring[i]);
end    

In [41]:
dfRef

Row,Metabolite,MetaboliteID,StandardizedName,Oxidized,OH,O,O₂,O₃,O₄,Plasmanyl,Plasmalogen,CHO,Ke
,String,String,String,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,CE(18:1),MT10001,CE(18:1),false,false,false,false,false,false,false,false,false,false
2,CE(18:2),MT10002,CE(18:2),false,false,false,false,false,false,false,false,false,false
3,CE(18:3),MT10003,CE(18:3),false,false,false,false,false,false,false,false,false,false
4,CE(18:4),MT10004,CE(18:4),false,false,false,false,false,false,false,false,false,false
5,CE(20:1),MT10005,CE(20:1),false,false,false,false,false,false,false,false,false,false
6,CE(20:2),MT10006,CE(20:2),false,false,false,false,false,false,false,false,false,false
7,CE(20:4),MT10007,CE(20:4),false,false,false,false,false,false,false,false,false,false
8,CE(20:5),MT10008,CE(20:5),false,false,false,false,false,false,false,false,false,false
9,CE(22:3),MT10009,CE(22:3),false,false,false,false,false,false,false,false,false,false


In [42]:
dfClassification = fetch_properties(dfRef.StandardizedName);
insertcols!(dfClassification, 1, :Metabolite => dfRef.Metabolite);
first(dfClassification, 3)

Row,Metabolite,exactmass,formula,main_class,refmet_name,sub_class,super_class
,String,String?,String?,String?,String?,String?,String?
1,CE(18:1),650.6002,C45H78O2,Sterol esters,CE 18:1,Chol. esters,Sterol Lipids
2,CE(18:2),648.5845,C45H76O2,Sterol esters,CE 18:2,Chol. esters,Sterol Lipids
3,CE(18:3),646.5689,C45H74O2,Sterol esters,CE 18:3,Chol. esters,Sterol Lipids


In [43]:
# replace "-" by missing
for c ∈ eachcol(dfClassification)
           replace!(c, "-" => missing)
end

In [44]:
idxmissing = findall(ismissing.(dfClassification.main_class))
dfRef.StandardizedName[idxmissing]

11-element Vector{String}:
 "PC(22:2_16:3)"
 "TG(16:0_22:6_22:3)"
 "TG(18:0_22:6_22:3)"
 "PC(20:1/16:0)"
 "PMe(16:0/18:1)"
 "TG(18:2/16:0/17:0)"
 "TG(18:2/17:0/22:5)"
 "TG(18:3/20:0/20:0)"
 "TG(22:5/22:4/22:6)"
 "TG(22:6/18:1/19:0)"
 "ZyE(22:5)"

At this stage, only *DMPE(16:0_22:6)* can be processed by [goslin](https://apps.lifs-tools.org/goslin/).   
We will filter *PMe(16:0/18:1)* (phosphatidylmethanol) [5] and *ZyE(22:5)*.


In [45]:
dfRef |> names

13-element Vector{String}:
 "Metabolite"
 "MetaboliteID"
 "StandardizedName"
 "Oxidized"
 "OH"
 "O"
 "O₂"
 "O₃"
 "O₄"
 "Plasmanyl"
 "Plasmalogen"
 "CHO"
 "Ke"

In [46]:
dfClassification |> names

7-element Vector{String}:
 "Metabolite"
 "exactmass"
 "formula"
 "main_class"
 "refmet_name"
 "sub_class"
 "super_class"

In [47]:
dfRef = leftjoin(dfRef, dfClassification, on = :Metabolite); size(dfRef)

(590, 19)

In [48]:
# filter
# deleteat!(dfRef, idxmissing[[2,3]]);
deleteat!(dfRef, idxmissing[[5,11]]);

In [52]:
dfRef.sub_class |> unique

14-element Vector{Union{Missing, String}}:
 "Chol. esters"
 "Cer"
 "DAG"
 "PE-NMe2"
 "LPC"
 "LPE"
 "PC"
 missing
 "TAG"
 "O-PC"
 "PE"
 "PI"
 "O-PS"
 "SM"

In [57]:
?skipmissing

search: skipmissing skipmissings dropmissing ismissing dropmissing! idxmissing



```
skipmissing(itr)
```

Return an iterator over the elements in `itr` skipping [`missing`](@ref) values. The returned object can be indexed using indices of `itr` if the latter is indexable. Indices corresponding to missing values are not valid: they are skipped by [`keys`](@ref) and [`eachindex`](@ref), and a `MissingException` is thrown when trying to use them.

Use [`collect`](@ref) to obtain an `Array` containing the non-`missing` values in `itr`. Note that even if `itr` is a multidimensional array, the result will always be a `Vector` since it is not possible to remove missings while preserving dimensions of the input.

See also [`coalesce`](@ref), [`ismissing`](@ref), [`something`](@ref).

# Examples

```jldoctest
julia> x = skipmissing([1, missing, 2])
skipmissing(Union{Missing, Int64}[1, missing, 2])

julia> sum(x)
3

julia> x[1]
1

julia> x[2]
ERROR: MissingException: the value at index (2,) is missing
[...]

julia> argmax(x)
3

julia> collect(keys(x))
2-element Vector{Int64}:
 1
 3

julia> collect(skipmissing([1, missing, 2]))
2-element Vector{Int64}:
 1
 2

julia> collect(skipmissing([1 missing; 2 missing]))
2-element Vector{Int64}:
 1
 2
```


In [58]:
findall(occursin.("PE-NMe2", skipmissing(dfRef.sub_class)))

1-element Vector{Int64}:
 29

In [59]:
skipmissing(dfRef.sub_class)[29]

"PE-NMe2"

### Use GOSLIN

In [143]:
R"""
suppressMessages(library('rgoslin'))
suppressMessages(library('tidyverse'));
"""

RObject{StrSxp}
 [1] "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"     "readr"    
 [7] "tidyr"     "tibble"    "ggplot2"   "tidyverse" "rgoslin"   "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     


In [144]:
@rput dfRef;

In [145]:
R"""
# check validity
dfRef$Valid <- suppressWarnings(sapply(dfRef$StandardizedName, isValidLipidName))
if (sum(dfRef$Valid) == dim(dfRef)[1]) {
    cat("All valid.")
} else {
    print("Check invalid names.")
}
""";

All valid.

In [146]:
dfRef.Total_C = zeros(Int, size(dfRef,1));
dfRef.Total_DB = zeros(Int, size(dfRef,1));
dfRef.Class = repeat(["NA"], size(dfRef,1));

In [147]:
for i in 1:size(dfRef, 1)
    @rput i;
    R"""
    #rsltGoslin <- as_tibble(parseLipidNames(dfRef$StandardizedName[i]))[, c("Original Name", "Total C", "Total DB", "Lipid Maps Main Class")];
    rsltGoslin <- as_tibble(parseLipidNames(dfRef$StandardizedName[i]))[, c("Original.Name", "Total.C", "Total.DB", "Lipid.Maps.Main.Class")];
    """
    @rget rsltGoslin
    # dfRef.Total_C[i] = parse(Int, rsltGoslin."Total C"[1])
    # dfRef.Total_DB[i] = parse(Int, rsltGoslin."Total DB"[1])
    # dfRef.Class[i] = rsltGoslin."Lipid Maps Main Class"[1]
        
    dfRef.Total_C[i] = rsltGoslin."Total_C"[1]
    dfRef.Total_DB[i] = rsltGoslin."Total_DB"[1]
    dfRef.Class[i] = rsltGoslin."Lipid_Maps_Main_Class"[1]
end

In [148]:
names(dfRef)

22-element Vector{String}:
 "Metabolite"
 "MetaboliteID"
 "StandardizedName"
 "Oxidized"
 "OH"
 "O"
 "O₂"
 "O₃"
 "O₄"
 "Plasmanyl"
 "Plasmalogen"
 "CHO"
 "Ke"
 "exactmass"
 "formula"
 "main_class"
 "refmet_name"
 "sub_class"
 "super_class"
 "Total_C"
 "Total_DB"
 "Class"

#### Save metabolites reference dataset:

In [149]:
fileMetaboRef = joinpath(@__DIR__,"..","..","data","processed","refMeta.csv");
dfRef |> CSV.write(fileMetaboRef);

### Sub class dictionary

In [150]:
fileClinicalDict = joinpath(@__DIR__,"..","..","data","processed", "SubClassDictionary.csv");
open(fileClinicalDict,"w") do io
   println(io,
        "SubClass,Name\n",
        "Cer,Ceramides\n",
        "Chol. esters,Cholesteryl esters\n",
        "DAG,Diglycerides\n",
        "LPC,Lysophosphatidylcholines\n",
        "LPE,Lysocephalins\n",
        "PC,Phosphatidylcholines\n",
        "PE,Phosphatidylethanolamines\n" ,
        "PI,Phosphatidylinositols\n",
        "PS,Phosphatidylserines\n",
        "SM,Sphingomyelins\n",
        "TAG,Triglycerides\n"
    )
end

## Extract Metabolites dataset 

In [151]:
dfMetabo = fetch_data(ST);

In [152]:
# rename sample ID with suffix
vHeader = names(dfMetabo);
vHeader[2:end] .= "ID_".*vHeader[2:end];
rename!(dfMetabo, Symbol.(vHeader));

In [153]:
first(dfMetabo, 5)

Row,Metabolite,ID_8358,ID_8363,ID_8370,ID_8371,ID_8373,ID_8376,ID_8378,ID_8379,ID_8385,ID_8390,ID_8392,ID_8393,ID_8399,ID_8403,ID_8404,ID_8356,ID_8382,ID_8387,ID_8391,ID_8394,ID_8396,ID_8405,ID_8407,ID_8359,ID_8360,ID_8367,ID_8368,ID_8372,ID_8374,ID_8375,ID_8377,ID_8380,ID_8383,ID_8384,ID_8395,ID_8397,ID_8400,ID_8402,ID_8355,ID_8362,ID_8364,ID_8366,ID_8381,ID_8386,ID_8389,ID_8398,ID_8401,ID_8406,ID_8408,ID_8409,ID_8410
,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String
1,CE(18:1),1005.004533,656.2196475,442.532242,835.7863174,694.9938797,1113.26766,601.1531229,911.0231408,922.4312376,2162.927368,2662.061708,2554.95758,317.2679899,954.8437234,1485.162418,1487.098997,1526.877146,2350.745891,689.0949408,2108.859738,2120.099614,3248.33437,1792.058802,819.9110147,822.0304468,1102.919542,995.0302335,835.9690242,668.6597227,589.6627367,649.3858632,1133.170376,441.4649634,725.1292872,954.0964175,788.8463418,666.9293928,492.7900539,2136.677671,1067.73527,1039.51039,962.4501876,1967.906092,463.6840503,747.2729114,1187.124441,1413.081456,2137.14665,935.8754413,1314.455617,994.0309683
2,CE(18:2),120.5422615,81.57473155,59.37249758,95.26536816,111.2627008,104.0151326,55.54637576,77.51145235,76.82468683,175.1010307,198.8355773,340.6354778,28.50615715,88.36516868,98.43064945,144.5494583,102.7142211,290.1930501,72.614949,156.1764691,165.3048481,212.0992928,127.0272866,104.0060156,71.89492666,126.7917354,130.1923503,99.16116764,69.02562776,45.094948,63.01183159,131.2795755,41.78108707,89.24603905,137.5258398,97.81887033,82.5493078,74.18123187,299.7572817,79.35524902,80.02628309,124.415973,135.3763366,59.25467281,98.15699392,186.498727,59.23789656,155.9713913,80.41659849,130.579997,94.47065133
3,CE(18:3),207.5760181,103.0388756,91.39644092,147.2090585,134.3577342,163.0631881,87.38702958,116.6200796,145.1578368,249.2077712,414.5426793,384.6989707,53.82494811,156.2226748,271.1422424,240.1673799,181.7064279,515.154009,112.2510577,319.6809932,464.6252606,595.9820339,253.3709138,111.1392527,112.7036474,159.380235,207.1645841,184.2719717,93.31381194,106.8488859,89.68337726,139.0250959,57.01127039,179.7445219,185.3427762,165.1674202,149.8846796,103.7963029,327.7022204,173.4720418,191.2745002,243.4699394,311.159407,77.98130404,157.4007537,256.5046666,168.9243624,276.7940302,122.7791609,301.2520366,204.2388073
4,CE(18:4),185.1900835,114.8704705,102.3479244,214.2161794,105.0702351,210.1147451,119.3918239,199.4102296,258.127771,400.4438389,313.5091499,673.4791258,139.3275734,140.5671856,313.1666872,321.532862,323.6841529,587.0712331,161.7361894,467.6293789,462.586779,643.1870656,395.1143264,148.5243146,186.0612789,117.8878366,281.9749001,270.8238429,208.7471171,207.2171458,281.7002965,140.7439699,65.61017035,229.4907776,239.8230071,210.9786658,189.1752974,155.7311868,216.8449078,291.7592911,220.9522036,235.8758164,420.4788756,116.8234124,220.9994502,310.6566702,195.3825583,294.1042009,222.0598007,395.8092463,263.3779846
5,CE(20:1),27.12701689,2.961702936,4.237597992,31.33287865,7.637112491,17.99623889,5.5013182,13.16834995,8.599539077,34.25623205,85.12014992,100.5495999,0,22.8589363,87.21672373,49.25723513,19.65817783,238.4575132,13.75900937,87.75320082,113.7836729,231.6847272,77.78389937,0,14.86124058,44.68949221,10.97565362,13.16168034,6.927717933,0,0,12.10309837,0,14.06325594,14.23398122,6.91985563,8.038768426,0,44.75595046,31.9501526,57.53215516,38.2939869,95.1415906,0,4.24477895,21.08378352,78.52368786,212.7656602,14.78322598,77.8490411,39.61243763


Replace `Metabolite` name information with `MetaboliteID` values:

In [154]:
dfMetaboAll = leftjoin(select(dfRefOriginal, [:Metabolite, :MetaboliteID]), dfMetabo, on = [:Metabolite]);
select!(dfMetaboAll, Not([:Metabolite]));

Select the samples that only present in the filtered clinical dataset, `dfIndividuals`:   

In [155]:
select!(dfMetaboAll, vcat([:MetaboliteID], Symbol.(dfIndividuals.SampleID)));
size(dfMetaboAll)

(590, 45)

#### Save metabolites levels dataset:

In [156]:
fileMetabo = joinpath(@__DIR__,"..","..","data","processed","Metabo.csv");
dfMetaboAll = permutedims(dfMetaboAll, 1, :SampleID);
dfMetaboAll |> CSV.write(fileMetabo);

## References

[1] Koelmel, J. P., Ulmer, C. Z., Fogelson, S., Jones, C. M., Botha, H., Bangma, J. T., Guillette, T. C., Luus-Powell, W. J., Sara, J. R., Smit, W. J., Albert, K., Miller, H. A., Guillette, M. P., Olsen, B. C., Cochran, J. A., Garrett, T. J., Yost, R. A., & Bowden, J. A. (2019). Lipidomics for wildlife disease etiology and biomarker discovery: a case study of pansteatitis outbreak in South Africa. Metabolomics : Official journal of the Metabolomic Society, 15(3), 38. https://doi.org/10.1007/s11306-019-1490-9    

[2] Kato, S., Shimizu, N., Hanzawa, Y., Otoki, Y., Ito, J., Kimura, F., Takekoshi, S., Sakaino, M., Sano, T., Eitsuka, T., Miyazawa, T., & Nakagawa, K. (2018). Determination of triacylglycerol oxidation mechanisms in canola oil using liquid chromatography-tandem mass spectrometry. NPJ science of food, 2, 1. https://doi.org/10.1038/s41538-017-0009-x    

[3] Koelmel, J. P., Ulmer, C. Z., Jones, C. M., Yost, R. A., & Bowden, J. A. (2017). Common cases of improper lipid annotation using high-resolution tandem mass spectrometry data and corresponding limitations in biological interpretation. Biochimica et biophysica acta. Molecular and cell biology of lipids, 1862(8), 766–770. https://doi.org/10.1016/j.bbalip.2017.02.016    

[4] Riewe, D., Wiebach, J., & Altmann, T. (2017). Structure Annotation and Quantification of Wheat Seed Oxidized Lipids by High-Resolution LC-MS/MS. Plant physiology, 175(2), 600–618. https://doi.org/10.1104/pp.17.00470    

[5] Koelmel, J. P., Jones, C. M., Ulmer, C. Z., Garrett, T. J., Yost, R. A., Schock, T. B., & Bowden, J. A. (2018). Examining heat treatment for stabilization of the lipidome. Bioanalysis, 10(5), 291–305. https://doi.org/10.4155/bio-2017-0209    

